In [19]:
# IMPORTS

# Standard library imports
import argparse
import json  # For reading JSON configuration files

# Third-party library imports
import numpy as np  # Array handling
import pandas as pd  # Data manipulation
import matplotlib.pyplot as plt  # Plotting
from matplotlib.colors import ListedColormap, LogNorm  # Custom color mapping and log scaling
from matplotlib.colors import Normalize
from mpl_toolkits.basemap import Basemap  # Mapping functions
import matplotlib.patches as mpatches  # For custom legend patches

# Local module imports
from fetch_map_df import fetch_map_df  # Function to fetch data with bounding boxes
from basemap_imshow_discrete_or_heatmap import basemap_imshow_discrete_or_heatmap  # Custom mapping function

# Assigning parsed arguments to variables
plotting_parameters = ["predicted_ecosystem"]
level = 3
year = 2012
region = "African wall"
silent = False
color = "custom"
scale = "linear"

# Load custom colors from JSON
if color == "custom":
    with open("../scripts/custom_colors.json", "r") as json_file:
        color = json.load(json_file)  # Dictionary for custom color mapping

#START SCRIPT
#fetch dataframe
df = fetch_map_df(year = year, region=region, plotting_parameters=plotting_parameters)

#modify dataframe if necessary
if "predicted_ecosystem" in df.columns:
    df[['ecosystem_level_1', 'ecosystem_level_2', 'ecosystem_level_3']] = df['predicted_ecosystem'].str.split('-', expand=True)
    df["ecosystem_level_3"] = df["ecosystem_level_2"]+"-"+df["ecosystem_level_3"]
    if level == 1:
        plotting_col = "ecosystem_level_1"
    elif level == 2:
        plotting_col = "ecosystem_level_2"
    elif level == 3:
        plotting_col = "ecosystem_level_3"
    else:
        plotting_col = "predicted_ecosystem"
else:
    plotting_col=plotting_parameters[0]   



--- DB SIGN IN: ---
reading .env files...
>> Connected to: postgres
>> Schemas available: ecosystem_classifier, ecosystem_outputs, ecosystem_predictions, ecosystem_webapp, information_schema, public
>> "Output" tables in "ecosystem_outputs": [15] ecosystems_2015_output, ecosystems_2030_ ...

Fetching region "African wall" and parameters ""predicted_ecosystem", "lat", "lon"" from "ecosystems_2012_output"

SUCCESS: fetched 40817 records for "predicted_ecosystem, lat, lon"


In [20]:
#basemap_imshow_discrete_or_heatmap(
lat=df["lat"]#, 
lon=df["lon"]#, 
plotting_col=df[plotting_col]#, 
lat_min=df['lat'].min()#, 
lat_max=df['lat'].max()#, 
lon_min=df['lon'].min()#, 
lon_max=df['lon'].max()#,
title=plotting_parameters[0]#,
scale=scale,
color=color
#)

In [21]:
# Calculate aspect ratio based on latitude and longitude bounding box
aspect_ratio = abs((lat_max - lat_min) / (lon_max - lon_min))
fig_width = 10  # Set a base width
fig_height = fig_width * aspect_ratio  # Adjust height to match aspect ratio
# Create a DataFrame for pivoting to a consistent grid
data = pd.DataFrame({'lat': lat, 'lon': lon, 'value': plotting_col})
# Determine if values are numeric or categorical (strings)
if pd.api.types.is_numeric_dtype(plotting_col):
    # If numeric, pivot the data and apply a heatmap colormap
    if color == "default":
        cmap = "viridis"
    else:
        cmap = color  
    
    pivoted_data = data.pivot(index='lat', columns='lon', values='value')
    
    # Apply logarithmic scaling if specified
    if scale == "log":
        vmin = pivoted_data[pivoted_data > 0].min().min()
        norm = LogNorm(vmin=vmin, vmax=pivoted_data.max().max())
    else:
        norm = None  # Linear scale

else:
    # Categorical data handling remains the same
    unique_values = plotting_col.dropna().unique()
    value_to_int = {val: i for i, val in enumerate(unique_values)}
    data['value_int'] = data['value'].map(value_to_int)
    #color mapping
    if color == "default":
        cmap = ListedColormap(plt.cm.get_cmap('tab20').colors[:len(unique_values)])
    else:
        if color in plt.colormaps():
            #return 
            print("!Warning!:\t color your provided appears to be a heatmap colorscale\n\tnot applicable for discrete values")
        else:
            color = [color[val] for val in unique_values if val in color]
            cmap = ListedColormap(color)
    pivoted_data = data.pivot(index='lat', columns='lon', values='value_int')
    norm = None  # Norm not needed for categorical data

# Ensure the latitude and longitude grid arrays are aligned with the pivoted data
lat_grid = pivoted_data.index.values
lon_grid = pivoted_data.columns.values
int_col_reshaped = pivoted_data.values  # This now contains NaN for missing values
# Set up figure with dynamically calculated size
fig, ax = plt.subplots(figsize=(fig_width, fig_height))

# Flexible Basemap projection with dynamic bounding box
m = Basemap(
    projection='cyl',  # Using cylindrical projection for better alignment
    llcrnrlat=lat_min, urcrnrlat=lat_max, 
    llcrnrlon=lon_min, urcrnrlon=lon_max, 
    resolution='i', ax=ax
)
#m.drawcoastlines()
m.drawmapboundary(fill_color='lightblue',zorder=0)  # Ocean color
m.fillcontinents(color='lightgrey', lake_color='lightblue', zorder=1)  # Continent color, lake color to match ocean
# Transforming lat-lon to Basemap coordinates and plotting
x, y = np.meshgrid(lon_grid, lat_grid)
c = ax.imshow(int_col_reshaped, extent=[lon_min, lon_max, lat_min, lat_max], 
              origin='lower', cmap=cmap, aspect='auto', zorder=2, norm=norm)
# Add legend for categorical data
if not pd.api.types.is_numeric_dtype(plotting_col):
    handles = [mpatches.Patch(color=cmap(i), label=val) for i, val in enumerate(unique_values)]
    plt.legend(handles=handles, title="Categories", bbox_to_anchor=(1.05, 1), loc='upper left')

if title == "default":
    plt.title('Basemap with Heatmap or Discrete Values')
else: 
    plt.title(title+"_"+str(year))
plt.colorbar(c, ax=ax, label='Value') if pd.api.types.is_numeric_dtype(plotting_col) else None
#plt.show()
# Save the plot as a PNG file
plt.savefig("predicted"+str(year)+".png", format="png", dpi=300)  # Save with desired resolution

# Optionally, clear the plot if needed
plt.close()